In [1]:
import numpy as np
import pandas as pd

# Load SPFM data and conver it to Panda dataframe

In [2]:
%%sh
wget http://www.philippe-fournier-viger.com/spmf/datasets/SIGN.txt

--2020-10-28 12:16:47--  http://www.philippe-fournier-viger.com/spmf/datasets/SIGN.txt
Resolving www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)... 74.208.236.167
Connecting to www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)|74.208.236.167|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236727 (231K) [text/plain]
Saving to: ‘SIGN.txt’

     0K .......... .......... .......... .......... .......... 21%  136K 1s
    50K .......... .......... .......... .......... .......... 43%  271K 1s
   100K .......... .......... .......... .......... .......... 64%  502K 0s
   150K .......... .......... .......... .......... .......... 86%  597K 0s
   200K .......... .......... .......... .                    100%  822K=0.8s

2020-10-28 12:16:49 (299 KB/s) - ‘SIGN.txt’ saved [236727/236727]



In [3]:
%%sh
wget http://www.philippe-fournier-viger.com/spmf/datasets/kosarak_sequences.txt

--2020-10-28 12:25:52--  http://www.philippe-fournier-viger.com/spmf/datasets/kosarak_sequences.txt
Resolving www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)... 74.208.236.167
Connecting to www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)|74.208.236.167|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60046520 (57M) [text/plain]
Saving to: ‘kosarak_sequences.txt.1’

     0K .......... .......... .......... .......... ..........  0%  136K 7m11s
    50K .......... .......... .......... .......... ..........  0%  272K 5m23s
   100K .......... .......... .......... .......... ..........  0%  501K 4m14s
   150K .......... .......... .......... .......... ..........  0%  597K 3m35s
   200K .......... .......... .......... .......... ..........  0%  852K 3m5s
   250K .......... .......... .......... .......... ..........  0% 1.03M 2m44s
   300K .......... .......... .......... .......... ..........  0% 1.28M 2m26s
   350K .......... .

In [4]:
spfm_data='kosarak_sequences.txt'

In [5]:
#Read SPFM raw data
f = open(spfm_data, 'r+')
raw_lines = [line.strip() for line in f.readlines()]
f.close()

In [6]:
#Seperator is -1 and -2 is end of line -->convert into CSV 
itemset_dict={}
i=0
max_cols=0
for line in raw_lines:
  itemset=[int(item.strip()) for item in line.split('-1')[:-1] ]
  itemset_dict[i]=itemset
  if max_cols < len(itemset):
    max_cols=len(itemset)
  i=i+1

In [ ]:
#Make each row of same number of columns by padding zeros upto max_columns
from itertools import repeat
for key in itemset_dict:
  itemset_dict[key].extend(repeat('?', max_cols - len(itemset_dict[key])))

In [ ]:
len(itemset_dict[1])

In [ ]:
itemset_list=[]
for key in itemset_dict:
  itemset_list.append(itemset_dict[key])

In [ ]:
df=pd.DataFrame(np.array(itemset_list))

df.head()

In [ ]:
my_corpus=[]
for line in raw_lines:
  itemset=[int(item.strip()) for item in line.split('-1')[:-1] ]
  my_string = ','.join(str(x) for x in itemset)
  my_corpus.append(my_string)

**Itemsets of all transactions together**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(my_corpus)
print(vectorizer.get_feature_names())

In [ ]:
df=pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
df.head()

In [ ]:
df[df==0] = '?'
df[df==1] ='t'

In [ ]:
df.head()

**Create 'arff' format suitbale to load in WEKA**

In [1]:
import shutil
import os

att_file=open('Header.txt','w')
att_file.write("@relation    "  + spfm_data.split('.')[0])
att_file.write('\n')

for att in vectorizer.get_feature_names():
  att_new='\''+att+'\''
  my_str='@attribute '+att_new+ ' { t}'
  att_file.write(my_str)
  att_file.write('\n')
att_file.write('@data')
att_file.write('\n')  
att_file.close()  

df.to_csv('Data.csv',index=False,header=False)

target_filename=spfm_data.split('.')[0]+".arff"


with open(target_filename,'wb') as wfd:
    for f in ['Header.txt','Data.csv']:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd)
os.remove('Header.txt')  
os.remove('Data.csv')    

NameError: ignored

In [ ]:
from google.colab import files
files.download(target_filename)